In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
from scipy import signal

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold, GridSearchCV  , StratifiedGroupKFold, cross_validate
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier

from random import sample, shuffle
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone
from copy import deepcopy

import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
dataset_path_root = "/home/bruno/Academico/Doctorado/Neuro_Fairness/Shu_Dataset/002_Dataset_Traditional_Feature_Extraction/"
participants=["sub-001","sub-002","sub-003","sub-004","sub-005",
              "sub-006","sub-007","sub-008","sub-009","sub-010",
              "sub-011","sub-012","sub-013","sub-014","sub-015",
              "sub-016","sub-017","sub-018","sub-019","sub-020",
              "sub-021","sub-022","sub-023","sub-024","sub-025"]
sessions = ["ses-01","ses-02","ses-03","ses-04","ses-05"]

In [7]:
dataset={}
for participant in participants:
    dataset[participant]={}
    data_path=participant+"_task_motorimagery_eeg_preprocessing_band_power_features.mat"
    data=sio.loadmat(dataset_path_root + data_path)
    for session in sessions:
        dataset[participant][session +'_data_band_power']=data[session +'_data_band_power']
        dataset[participant][session +'_labels_trials']=data[session +'_labels_trials']
    dataset[participant]['sfreq']=np.squeeze(data['sfreq'])
    dataset[participant]['age']=np.squeeze(data['age'])
    dataset[participant]['gender']=data['gender'][0]
    dataset[participant]['group_medidator']=data['group_medidator'][0]
    dataset[participant]['id_participant']=data['id_participant'][0]

In [8]:
index_female = []
index_male = []
for participant in participants:
    if dataset[participant]['gender'] == 'M':
        index_male.append(participant)
    elif dataset[participant]['gender'] == 'F':
        index_female.append(participant)
print(f"Participantes hombres: {index_male}")
print(f"Participantes mujeres: {index_female}")

Participantes hombres: ['sub-001', 'sub-002', 'sub-008', 'sub-012', 'sub-013', 'sub-015', 'sub-017', 'sub-018', 'sub-019', 'sub-021', 'sub-022', 'sub-023', 'sub-025']
Participantes mujeres: ['sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-007', 'sub-009', 'sub-010', 'sub-011', 'sub-014', 'sub-016', 'sub-020', 'sub-024']


In [9]:
for participant in participants:
    data_ = np.concatenate((dataset[participant]['ses-01_data_band_power'],
                            dataset[participant]['ses-02_data_band_power'],
                            dataset[participant]['ses-03_data_band_power'],
                            dataset[participant]['ses-04_data_band_power'],
                            dataset[participant]['ses-05_data_band_power']),axis=0)
    
    dataset[participant]['data_band_power'] = data_
    dataset[participant]['data_gender'] =  np.array(list(dataset[participant]['gender']) * data_.shape[0])
    dataset[participant]['group_participant'] =  np.array(list([participant]) * data_.shape[0])

In [46]:



N_it_rand = 100
n_feature_rand = 6
info_rand_exps = {}
for it_rand in range(N_it_rand):
    # Seleccionamos aleatoriamente 6 caracteristicas de cada sujeto que iran cambiando en cada iteración 
    for participant in participants:
        X = dataset[participant]['data_band_power'].copy()
        n_0 = X.shape[0]
        n_1 = X.shape[1]
        n_2 = X.shape[2]
        X = X.reshape(n_0,n_1*n_2)
        X_random = np.zeros((n_0,n_feature_rand))
        for it0 in range(n_0):
            X_random[it0,:]=sample(X[it0,:].tolist(), n_feature_rand)    
        dataset[participant]['it_'+str(it_rand)+'_data_random'] = np.array(X_random)

    n_features = data_.shape[-1]
    N_it = 20
    n_test_participant = 2
    n_val_participant = 2 
    n_ign_participant = 1

    max_iter = 200
    patience = 50
    info_exp = {}
    info_clfs = {}
    #Comenzamos con el proceso de entrenamiento de los sujetos
    for it in range(N_it):
        dic_aux = {}
        
        X_train = None
        X_val = None
        X_test = None 
        
        X_train_ = None
        X_val_ = None
        X_test_ = None
        
        idx_male = index_male.copy()
        idx_female = index_female.copy()
        
        # TEST PARTICIPANTS:
        idx_male_test = sample(idx_male, n_test_participant)
        idx_female_test = sample(idx_female, n_test_participant)
        for it_ in range(n_test_participant):
            idx_male.remove(idx_male_test[it_])
            idx_female.remove(idx_female_test[it_])
        idx_test = idx_male_test + idx_female_test
        dic_aux['reg_idx_test'] = idx_test   
        
        # VALIDATION PARTICIPANTS:
        idx_male_val = sample(idx_male, n_val_participant)
        idx_female_val = sample(idx_female, n_val_participant)
        for it_ in range(n_val_participant):
            idx_male.remove(idx_male_val[it_])
            idx_female.remove(idx_female_val[it_])
        idx_val = idx_male_val + idx_female_val
        dic_aux['reg_idx_val'] = idx_val  
        
        # TRAIN PARTICIPANTS:
        idx_male_ignore = sample(idx_male, n_ign_participant)
        for it_ in range(n_ign_participant):
            idx_male.remove(idx_male_ignore[it_])
        idx_male_train = idx_male.copy()
        idx_female_train = idx_female.copy()
        idx_train = idx_male_train + idx_female_train
        dic_aux['reg_idx_train'] = idx_train    
            
        
        # CONCATENAMOS EL CONJUNTO DE DATOS
        # TEST
        X_test = np.zeros((1,n_features))
        Y_test = np.zeros(1)
        for participant in idx_test:
            X_test = np.concatenate((X_test, dataset[participant]['it_'+str(it_rand)+'_data_random']),axis=0)
            Y_test = np.concatenate((Y_test, dataset[participant]['data_gender']),axis=0)
        X_test = X_test[1:,:]
        Y_test = Y_test[1:]
        dic_aux['n_trials_test'] = {'male':np.sum(Y_test == 'M'),'female':np.sum(Y_test == 'F')}
        dic_aux['proportion_trials_test'] = {'male':np.sum(Y_test == 'M')/(np.sum(Y_test == 'M')+np.sum(Y_test == 'F')),
                                        'female':np.sum(Y_test == 'F')/(np.sum(Y_test == 'M')+np.sum(Y_test == 'F'))}
        # VALIDATION
        X_val = np.zeros((1,n_features))
        Y_val = np.zeros(1)
        for participant in idx_val:
            X_val = np.concatenate((X_val, dataset[participant]['it_'+str(it_rand)+'_data_random']),axis=0)
            Y_val = np.concatenate((Y_val, dataset[participant]['data_gender']),axis=0)
        X_val = X_val[1:,:]
        Y_val = Y_val[1:]
        dic_aux['n_trials_val'] = {'male':np.sum(Y_val == 'M'),'female':np.sum(Y_val == 'F')}
        dic_aux['proportion_trials_val'] = {'male':np.sum(Y_val == 'M')/(np.sum(Y_val == 'M')+np.sum(Y_val == 'F')),
                                        'female':np.sum(Y_val == 'F')/(np.sum(Y_val == 'M')+np.sum(Y_val == 'F'))}
            
        # TRAIN
        X_train = np.zeros((1,n_features))
        Y_train = np.zeros(1)
        for participant in idx_train:
            X_train = np.concatenate((X_train, dataset[participant]['it_'+str(it_rand)+'_data_random']),axis=0)
            Y_train = np.concatenate((Y_train, dataset[participant]['data_gender']),axis=0)
        X_train = X_train[1:,:]
        Y_train = Y_train[1:]
        dic_aux['n_trials_train'] = {'male':np.sum(Y_train == 'M'),'female':np.sum(Y_train == 'F')}
        dic_aux['proportion_trials_train'] = {'male':np.sum(Y_train == 'M')/(np.sum(Y_train == 'M')+np.sum(Y_train == 'F')),
                                        'female':np.sum(Y_train == 'F')/(np.sum(Y_train == 'M')+np.sum(Y_train == 'F'))}   
        
        info_exp[f'it_{it}']=dic_aux
        
        dic_aux={}
        
        #-----------------------CLASSIFIER 1 -----------------------------#
        acc_train = None
        acc_val = None
        acc_test = None
        
        tol = 1e-4
        scaler = StandardScaler()
        clf = LinearDiscriminantAnalysis(tol=tol)
        
        X_train_ = scaler.fit_transform(X_train, Y_train)
        X_val_ = scaler.transform(X_val)
        X_test_ = scaler.transform(X_test)
        
        
        clf.fit(X_train_, Y_train)
        acc_train = clf.score(X_train_ , Y_train)
        acc_val = clf.score(X_val_ , Y_val)
        acc_test = clf.score(X_test_, Y_test)
        
        dic_aux['clf_1']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
        info_clfs[f'it_{it}']=dic_aux
        
    info_rand_exps[f'random_sample_{it_rand}'] = {'info_clfs': info_clfs , 'info_exp': info_exp}

In [47]:
#Generamos la tabla de los acc para cada iteración y clasificador
n_clfs = 1

matrix_acc_train = np.zeros((N_it_rand,n_clfs,N_it))
matrix_acc_val = np.zeros((N_it_rand,n_clfs,N_it))
matrix_acc_test = np.zeros((N_it_rand,n_clfs,N_it))

for it_rand in range(N_it_rand): 
    for it1 in range(n_clfs):
        for it0 in range(N_it):
            matrix_acc_train[it_rand,it1,it0] = info_rand_exps[f'random_sample_{it_rand}']['info_clfs'][f'it_{it0}'][f'clf_{it1+1}']['acc_train']
            matrix_acc_val[it_rand,it1,it0] = info_rand_exps[f'random_sample_{it_rand}']['info_clfs'][f'it_{it0}'][f'clf_{it1+1}']['acc_val']
            matrix_acc_test[it_rand,it1,it0] = info_rand_exps[f'random_sample_{it_rand}']['info_clfs'][f'it_{it0}'][f'clf_{it1+1}']['acc_test']

In [48]:
acc_train_clfs_mean = matrix_acc_train.mean(axis=2,keepdims=False)
acc_train_clfs_std = np.std(matrix_acc_train,axis=2,keepdims=False)

acc_val_clfs_mean = matrix_acc_val.mean(axis=2,keepdims=False)
acc_val_clfs_std = np.std(matrix_acc_val,axis=2,keepdims=False)

acc_test_clfs_mean = matrix_acc_test.mean(axis=2,keepdims=False)
acc_test_clfs_std = np.std(matrix_acc_test,axis=2,keepdims=False)

In [54]:
exps_random_list =  [f'random_sample_{it_rand}' for it_rand in range(N_it_rand)]
accs_train_df = pd.DataFrame(np.squeeze(matrix_acc_train),index=exps_random_list)
accs_val_df = pd.DataFrame(np.squeeze(matrix_acc_val),index=exps_random_list)
accs_test_df = pd.DataFrame(np.squeeze(matrix_acc_test),index=exps_random_list)
accs_test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
random_sample_0,0.380903,0.497054,0.531136,0.507405,0.515469,0.465855,0.442894,0.465128,0.462349,0.536885,0.426433,0.543684,0.623012,0.509732,0.578504,0.467717,0.565672,0.502343,0.592316,0.515767
random_sample_1,0.500000,0.514345,0.493886,0.491829,0.487138,0.510980,0.495996,0.563278,0.492047,0.546262,0.532167,0.556739,0.469430,0.535173,0.434089,0.643799,0.510109,0.463996,0.567869,0.542722
random_sample_2,0.459877,0.604156,0.555324,0.513247,0.497012,0.548793,0.497667,0.500000,0.568353,0.569755,0.557823,0.580240,0.444503,0.561348,0.454116,0.563801,0.619530,0.436583,0.476140,0.529289
random_sample_3,0.507878,0.592056,0.456961,0.509313,0.524142,0.513395,0.548705,0.455394,0.588963,0.517634,0.580343,0.573529,0.539537,0.367601,0.519168,0.505664,0.470315,0.547744,0.483218,0.534097
random_sample_4,0.509969,0.491357,0.469103,0.610735,0.534982,0.364583,0.506548,0.446130,0.484310,0.532835,0.612593,0.474774,0.499474,0.505670,0.460978,0.576803,0.567711,0.539530,0.597164,0.545360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
random_sample_95,0.442501,0.479666,0.574682,0.535733,0.561543,0.454876,0.568387,0.507758,0.560331,0.489440,0.497371,0.552859,0.553125,0.514493,0.445493,0.501832,0.475506,0.561792,0.492348,0.586207
random_sample_96,0.460145,0.406364,0.557802,0.530890,0.595645,0.479619,0.511278,0.528639,0.452899,0.534145,0.516535,0.460526,0.593880,0.523061,0.459764,0.479915,0.422954,0.570466,0.525226,0.428497
random_sample_97,0.470206,0.482059,0.518384,0.465455,0.404343,0.495065,0.641026,0.498446,0.557489,0.552824,0.454074,0.540773,0.480624,0.547631,0.499463,0.555732,0.493506,0.482415,0.548337,0.418058
random_sample_98,0.458422,0.470558,0.586702,0.488008,0.562967,0.505523,0.480302,0.585495,0.617143,0.444504,0.617428,0.607462,0.465453,0.520000,0.482456,0.503896,0.634555,0.550155,0.524948,0.488755


In [55]:
results_df = pd.DataFrame(np.concatenate((acc_train_clfs_mean,acc_train_clfs_std,
                                          acc_val_clfs_mean,acc_val_clfs_std,
                                          acc_test_clfs_mean,acc_test_clfs_std),axis=1),
                          columns = ['TRAIN MEAN', 'TRAIN STD','VAL MEAN', 'VAL STD','TEST MEAN', 'TEST STD']
                          ,index=exps_random_list)
results_df

,TRAIN MEAN,TRAIN STD,VAL MEAN,VAL STD,TEST MEAN,TEST STD
random_sample_0,0.530458,0.017821,0.539334,0.042135,0.506513,0.057389
random_sample_1,0.531759,0.017722,0.523889,0.057537,0.517593,0.044670
random_sample_2,0.529237,0.014525,0.496982,0.055215,0.526878,0.052781
random_sample_3,0.531574,0.017251,0.515173,0.056229,0.516783,0.052034
random_sample_4,0.531157,0.019084,0.512229,0.054170,0.516530,0.059232
...,...,...,...,...,...,...
random_sample_95,0.531612,0.017690,0.525247,0.059099,0.517797,0.044310
random_sample_96,0.536835,0.021398,0.509008,0.053274,0.501913,0.054001
random_sample_97,0.528791,0.014654,0.512503,0.038988,0.505296,0.053040
random_sample_98,0.527939,0.015236,0.537239,0.049268,0.529737,0.059029


In [56]:
from scipy.stats import ks_1samp
from scipy.stats import ttest_1samp
from scipy.stats import wilcoxon
from scipy import stats

In [60]:
alpha = 0.05
alpha_bonferroni = alpha / 100

test_norm = pd.DataFrame(columns=['statistics','p-value','H0'])
clf_names = accs_test_df.index.values.tolist()

for clf in clf_names: 
    acc_test = accs_test_df.loc[clf]
    norm_test = ks_1samp(acc_test,stats.norm.cdf)
    if norm_test.pvalue <= alpha_bonferroni:
        test_norm.loc[clf]=[norm_test.statistic,norm_test.pvalue,False]
    else:
        test_norm.loc[clf]=[norm_test.statistic,norm_test.pvalue,True]

print(f'Significancia: {alpha_bonferroni:.2}')
print(f'Intervalo de confianza: {100*(1-alpha_bonferroni)}')
test_norm

Significancia: 0.0005
Intervalo de confianza: 99.95


,statistics,p-value,H0
random_sample_0,0.648362,9.766425e-09,False
random_sample_1,0.667888,2.552574e-09,False
random_sample_2,0.668793,2.394531e-09,False
random_sample_3,0.643415,1.357322e-08,False
random_sample_4,0.642289,1.462025e-08,False
...,...,...,...
random_sample_95,0.670937,2.056859e-09,False
random_sample_96,0.657762,5.163683e-09,False
random_sample_97,0.657020,5.433454e-09,False
random_sample_98,0.671661,1.953524e-09,False


In [61]:
mu = 0.5
test = pd.DataFrame(columns=['statics','p-value','H0'])

for clf in clf_names: 
    acc_test = accs_test_df.loc[clf]
    w_test = wilcoxon(acc_test-mu)
    if w_test.pvalue <= alpha_bonferroni:
        test.loc[clf]=[w_test.statistic,w_test.pvalue,False]
    else: 
        test.loc[clf]=[w_test.statistic,w_test.pvalue,True]

print(f'Significancia: {alpha_bonferroni:.2}')
print(f'Intervalo de confianza: {100*(1-alpha_bonferroni)}')
test

/home/bruno/anaconda3/envs/fairness1.0/lib/python3.10/site-packages/scipy/stats/_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/bruno/anaconda3/envs/fairness1.0/lib/python3.10/site-packages/scipy/stats/_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/bruno/anaconda3/envs/fairness1.0/lib/python3.10/site-packages/scipy/stats/_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


Significancia: 0.0005
Intervalo de confianza: 99.95


/home/bruno/anaconda3/envs/fairness1.0/lib/python3.10/site-packages/scipy/stats/_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,statics,p-value,H0
random_sample_0,90.0,0.595819,True
random_sample_1,55.0,0.107466,True
random_sample_2,43.0,0.036385,True
random_sample_3,59.0,0.089695,True
random_sample_4,71.0,0.216167,True
...,...,...,...
random_sample_95,62.0,0.113987,True
random_sample_96,104.0,0.985435,True
random_sample_97,97.0,0.784126,True
random_sample_98,61.0,0.105398,True


In [62]:
test[test['H0'] == False]

,statics,p-value,H0
random_sample_29,17.0,0.000395,False
random_sample_68,7.0,0.000036,False
random_sample_82,7.0,0.000036,False
